In [ ]:
# Taking BP1 as an example (two variables: H10 and H100)
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

#  Input data (the sample plots from transfer area)
data = pd.read_excel("./trainSMGFfujianCOM.xlsx", header=None)
data_np = data.to_numpy()

#  Parameter Constraint Ranges (Two Variables)
param_bounds = {
    "weight_0": (-1.4234, -0.0906),   # H10  The numerical range depends on the parameter distribution of the model derived from your existing dataset.
    "weight_1": (4.98, 7.557),  # H100 
}

k = len(data_np)  #Sample size

# Dynamic Constraint Coefficient Function
def get_lambda_constraint(k, current_mse, max_lambda=30):
    if k < 5:
        return min(0.5 * current_mse, max_lambda)
    elif k <= 15:
        scale = (15 - k) / 10
        return min(0.5 * current_mse * scale, max_lambda)
    else:
        return 0.0

# Constraint Loss Function (2 weights)
def constraint_loss(model, lambda_constraint):
    reg_loss = 0
    w = model.weight[0]
    reg_loss += (torch.relu(w[0] - param_bounds["weight_0"][1]) + torch.relu(param_bounds["weight_0"][0] - w[0])) ** 2
    reg_loss += (torch.relu(w[1] - param_bounds["weight_1"][1]) + torch.relu(param_bounds["weight_1"][0] - w[1])) ** 2
    return lambda_constraint * reg_loss

# Build the Model
model = nn.Linear(2, 1)
model.weight = nn.Parameter(torch.tensor([[-0.757, 6.27]]))  # Initial parameter values of the baseline model
model.bias = nn.Parameter(torch.tensor([7.283]))  
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.1)
criterion = nn.MSELoss()

X = data_np[:, :2]        
y = data_np[:, 3]      
inputData = torch.from_numpy(X).float()
targetData = torch.from_numpy(y).float().view(-1, 1)

# Initial MSE and λ
with torch.no_grad():
    base_pred = model(inputData)
    base_mse = criterion(base_pred, targetData).item()
lambda_constraint = get_lambda_constraint(k, base_mse)

# MOdel training
model.train()
for epoch in range(2000):
    optimizer.zero_grad()
    pred = model(inputData)

    loss = criterion(pred, targetData)
    loss += constraint_loss(model, lambda_constraint)

    loss.backward()
    optimizer.step()
    scheduler.step()

# Output Model Parameters
final_weights = model.weight.detach().numpy().flatten()
final_bias = model.bias.detach().item()
print("Training completed. Model parameters are as follows:")
print(f"H10 = {final_weights[0]:.4f}")
print(f"H100 = {final_weights[1]:.4f}")
print(f"bias = {final_bias:.4f}")

